In [1]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from operator import itemgetter
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.pdf import PyMuPDFLoader

In [ ]:
Model = "llama3"; 
model = Ollama(model=Model)

In [ ]:
template = """The following is a conversation between a Human and and you (an AI Assistant). 
The Human will give a few documents as context and also his query.
You are always straight to the point and use only the details from the context given by the user and the conversation history. 
If you don't the answer to a question based on the previous conversation, you can say does not know.

Current conversation:
{history}

Human: {input}
AI Assistant:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

inputmessage = ""
context = ""


conversation = ConversationChain(
    prompt=PROMPT,
    llm=model,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant")
)

In [ ]:
loader = DirectoryLoader("testDataPDF",glob="*.pdf",loader_cls=PyMuPDFLoader)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [ ]:
db = FAISS.from_documents(docs, OllamaEmbeddings(model = "mxbai-embed-large"))
retriever = db.as_retriever()

In [ ]:
def combine(input, docs):

    context = "\n\n".join(f'Document [ Metadata : \n{doc.metadata} \nPage Content : \n{doc.page_content} ]' for doc in docs)
    combined = f"Here's some Context : \n {context} \n\n Here's my question : {input}"
    return combined

In [ ]:
def chat(question) : 
    docs = retriever.invoke(question)
    query = combine(question, docs)
    return conversation.predict(input = query)

In [ ]:
chat("What is the browser's high level structure?")